In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors

%matplotlib inline
# Use a valid Seaborn style available in Matplotlib styles
# You can check available styles with plt.style.available
plt.style.use('seaborn-v0_8-whitegrid')

Default Datset

Format
A data frame with 10000 observations on the following 4 variables.
	default
		A factor with levels No and Yes indicating whether the customer defaulted on their debt
	student
		A factor with levels No and Yes indicating whether the customer is a student
	balance
		The average balance that the customer has remaining on their credit card after making their monthly payment
	income 	Income of customer

In [ ]:
df = pd.read_excel('Default.xlsx')
df

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.625074
1,2,No,Yes,817.180407,12106.134700
2,3,No,No,1073.549164,31767.138947
3,4,No,No,529.250605,35704.493935
4,5,No,No,785.655883,38463.495879
...,...,...,...,...,...
9995,9996,No,No,711.555020,52992.378914
9996,9997,No,No,757.962918,19660.721768
9997,9998,No,No,845.411989,58636.156984
9998,9999,No,No,1569.009053,36669.112365


In [ ]:
df['default2'] = df.default.factorize()[0]
df['student2'] = df.student.factorize()[0]
df.head(3)

,Unnamed: 0,default,student,balance,income,default2,student2,default3,student3
0,1,No,No,729.526495,44361.625074,0,0,False,False
1,2,No,Yes,817.180407,12106.134700,0,1,False,True
2,3,No,No,1073.549164,31767.138947,0,0,False,False


In [ ]:
df.student.factorize()

(array([0, 1, 0, ..., 0, 0, 1]), Index(['No', 'Yes'], dtype='object'))

In [ ]:
X = df[['balance', 'income', 'student2']]
y = df.default2

In [ ]:
X

,balance,income,student2
0,729.526495,44361.625074,0
1,817.180407,12106.134700,1
2,1073.549164,31767.138947,0
3,529.250605,35704.493935,0
4,785.655883,38463.495879,0
...,...,...,...
9995,711.555020,52992.378914,0
9996,757.962918,19660.721768,0
9997,845.411989,58636.156984,0
9998,1569.009053,36669.112365,0


In [ ]:
y

,default2
0,0
1,0
2,0
3,0
4,0
...,...
9995,0
9996,0
9997,0
9998,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics as sm

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=15)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((8000, 3), (2000, 3), (8000,), (2000,))

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=7)
pred = knn.fit(X_train, y_train).predict(X_test)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
print(confusion_matrix(y_test, pred))

[[1926    8]
 [  57    9]]


In [ ]:
print(classification_report(y_test, pred, digits=3))

              precision    recall  f1-score   support

           0      0.971     0.996     0.983      1934
           1      0.529     0.136     0.217        66

    accuracy                          0.968      2000
   macro avg      0.750     0.566     0.600      2000
weighted avg      0.957     0.968     0.958      2000



In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_neighbors': [3, 5, 7, 9,11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn1 = neighbors.KNeighborsClassifier()
grid_search = GridSearchCV(knn1, param_grid)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 9, 11],
                         'weights': ['uniform', 'distance']})

In [ ]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)

Best Parameters: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'}
Best Cross-Validation Score: 0.9672500000000002


In [ ]:
pred = grid_search.predict(X_test)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
print(classification_report(y_test, pred, digits=3))

              precision    recall  f1-score   support

           0      0.970     0.997     0.983      1934
           1      0.545     0.091     0.156        66

    accuracy                          0.968      2000
   macro avg      0.758     0.544     0.570      2000
weighted avg      0.956     0.968     0.956      2000



In [ ]:
# prompt: want to create an application on the basis of this model that gives the default value as i put independent variable value , i want  default value in the enter custom infromation side

balance = 1500.0  #@param {type:"number"}
income = 40000.0  #@param {type:"number"}
is_student = "Yes" #@param ["Yes", "No"]

student_encoded = 1 if is_student == "Yes" else 0

# Prepare the input data in the same format as the training data
input_data = pd.DataFrame({
    'balance': [balance],
    'income': [income],
    'student2': [student_encoded]
})

# Make the prediction using the best model found by GridSearchCV
predicted_default_encoded = grid_search.predict(input_data)[0]

# Convert the prediction back to the original label ('Yes' or 'No')
predicted_default = 'Yes' if predicted_default_encoded == 1 else 'No'

print(f"Predicted default status: {predicted_default}")


Predicted default status: No
